# Dependencies

In [1]:
from sys import path
path.append('src/')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
from math import ceil
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from src.mrk_file import MRKFile
from src.iso_standard import PhotographicRequirements
from src.utils.plot import plot_confusion_matrix
from keras.applications import MobileNet
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model, Input

Using TensorFlow backend.


In [2]:
FILE_TRAIN_DUMP = 'data/train_dump.pkl'
FILE_TRAIN_BOTTLENECKS = 'data/train_bottlenecks.pkl'

FILE_VAL_DUMP = 'data/val_dump.pkl'
FILE_VAL_BOTTLENECKS = 'data/val_bottlenecks.pkl'

INPUT_SHAPE = (224, 224, 3)

In [3]:
output_names = ['blurred', 
                'looking_away', 
                'ink_marked_creased', 
                'unnatural_skin_tone', 
                'too_dark_light', 
                'washed_out', 
                'pixelation', 
                'hair_across_eyes', 
                'eyes_closed', 
                'varied_background', 
                'roll_pitch_yaw', 
                'flash_reflection_on_skin', 
                'red_eyes', 
                'shadows_behind_head', 
                'shadows_across_face', 
                'dark_tinted_lenses', 
                'flash_reflection_on_lenses', 
                'frames_too_heavy', 
                'frame_covering_eyes', 
                'hat_cap', 
                'veil_over_face', 
                'mouth_open', 
                'presence_of_other_faces_or_toys']

# Helper Methods 

In [4]:
def history_to_dataframe(history, epoch=-1):
    loss_train = np.array([history[name + '_loss'][epoch] for name in output_names])
    loss_val = np.array([history['val_' + name + '_loss'][epoch] for name in output_names])
    acc_train = np.array([history[name + '_acc'][epoch] for name in output_names]) * 100
    acc_val = np.array([history['val_' + name + '_acc'][epoch] for name in output_names]) * 100

    data = np.concatenate(([loss_train], [loss_val], [acc_train], [acc_val]), axis=0).T
    return pd.DataFrame(data, columns=['loss_train', 'loss_val', 'acc_train', 'acc_val'], index=output_names)

def load_proportions(file_txt):
    prop_values = np.loadtxt(file_txt)
    return pd.DataFrame(prop_values, index=output_names).T

def plot_graph(subplot, train, val, title):
    plt.subplot(*subplot)
    plt.plot(train, label='train')
    plt.plot(val, label='val')
    plt.xlabel('epoch')
    plt.title(title)
    plt.legend()

def save_history(history, output_file):
    n_epochs = len(history['loss'])
    n_graphs = len(output_names) * 2
    n_cols = 2
    n_rows = ceil(n_graphs / n_cols)

    df_train_props = load_proportions('data/train_max_proportions.txt')
    df_val_props = load_proportions('data/val_max_proportions.txt')
    
    plt.figure(figsize=(16, 100))
    for i, name in zip(range(1, n_graphs, 2), output_names):
        train_loss = history[name + '_loss']
        train_acc = history[name + '_acc']
        val_loss = history['val_' + name + '_loss']
        val_acc = history['val_' + name + '_acc']

        plot_graph((n_rows, n_cols, i), train_loss, val_loss, '{} ({})'.format(name, 'loss'))
        plot_graph((n_rows, n_cols, i + 1), train_acc, val_acc, '{} ({})'.format(name, 'acc'))
        plt.hlines(df_train_props[name], 0, n_epochs, linestyle=':', color='blue')
        plt.hlines(df_val_props[name], 0, n_epochs, linestyle=':', color='orange')

    plt.tight_layout()
    plt.savefig(output_file)
    plt.close()
    
def save_heatmaps(y_trues, y_preds, output_file, figsize=(20, 20)):
    plt.figure(figsize=figsize)

    n_graphs = len(output_names)
    n_cols = 5
    n_rows = ceil(n_graphs / n_cols)
    labels = np.unique(y_trues)

    for i, name, y_true, y_pred in zip(range(1, n_graphs + 1), output_names, y_trues, y_preds):
        y_pred = np.argmax(y_pred, axis=-1)
        conf_matrix = confusion_matrix(y_true.ravel(), y_pred, labels=labels)

        plt.subplot(n_rows, n_cols, i)
        plot_confusion_matrix(conf_matrix, 
                              target_names=['D', 'NC', 'C'], 
                              title=name, 
                              normalize=True)
    plt.savefig(output_file)
    plt.close()

# Load Data 

In [5]:
train_image_files, x_train, train_mrks = pkl.load(open(FILE_TRAIN_DUMP, 'rb'))

print(len(train_image_files), type(train_image_files))
print(x_train.shape, x_train.dtype)
print(len(train_mrks), type(train_mrks[0]))

5211 <class 'list'>
(5211, 224, 224, 3) float32
5211 <class 'src.mrk_file.MRKFile'>


In [6]:
val_image_files, x_val, val_mrks = pkl.load(open(FILE_VAL_DUMP, 'rb'))

print(len(val_image_files), type(val_image_files))
print(x_val.shape, x_val.dtype)
print(len(val_mrks), type(val_mrks[0]))

565 <class 'list'>
(565, 224, 224, 3) float32
565 <class 'src.mrk_file.MRKFile'>


# Save Bottlenecks 

In [7]:
base_model = MobileNet(input_shape=INPUT_SHAPE, weights='imagenet', include_top=False)

In [8]:
train_features = base_model.predict(x_train, batch_size=32, verbose=1)
val_features = base_model.predict(x_val, batch_size=32, verbose=1)

print(train_features.shape, train_features.dtype)
print(val_features.shape, val_features.dtype)

565/565 [==============================] - 1s 1ms/step
(5211, 7, 7, 1024) float32
(565, 7, 7, 1024) float32


In [9]:
train_values = np.array([mrk.photo_reqs.values() for mrk in train_mrks], dtype=np.int8)
val_values = np.array([mrk.photo_reqs.values() for mrk in val_mrks], dtype=np.int8)

print(train_values.shape, train_values.dtype)
print(val_values.shape, val_values.dtype)

(5211, 23) int8
(565, 23) int8


In [10]:
pkl.dump((train_features, train_values), open(FILE_TRAIN_BOTTLENECKS, 'wb'), protocol=-1)
pkl.dump((val_features, val_values), open(FILE_VAL_BOTTLENECKS, 'wb'), protocol=-1)

# Transfer Learning 

In [5]:
train_features, train_values = pkl.load(open(FILE_TRAIN_BOTTLENECKS, 'rb'))
val_features, val_values = pkl.load(open(FILE_VAL_BOTTLENECKS, 'rb'))

print(train_features.shape, train_features.dtype)
print(train_values.shape, train_values.dtype)
print(val_features.shape, val_features.dtype)
print(val_values.shape, val_values.dtype)

(5211, 7, 7, 1024) float32
(5211, 23) int8
(565, 7, 7, 1024) float32
(565, 23) int8


In [6]:
# encode reqs values from [-1, 0, 1] => [0, 1, 2]
enc = LabelEncoder()
enc.fit(train_values.ravel())

y_train = enc.transform(train_values.ravel()).reshape(train_values.shape)
y_val = enc.transform(val_values.ravel()).reshape(val_values.shape)

assert(y_train.shape == train_values.shape)
assert(y_val.shape == val_values.shape)
assert(np.all(y_train == train_values + 1))
assert(np.all(y_val == val_values + 1))

In [7]:
inputs = Input(shape=train_features.shape[1:], name='inputs')
avg_pool = GlobalAveragePooling2D()(inputs)

r2 = Dense(units=3, activation='softmax', name=output_names[0])(avg_pool)
r3 = Dense(units=3, activation='softmax', name=output_names[1])(avg_pool)
r4 = Dense(units=3, activation='softmax', name=output_names[2])(avg_pool)
r5 = Dense(units=3, activation='softmax', name=output_names[3])(avg_pool)
r6 = Dense(units=3, activation='softmax', name=output_names[4])(avg_pool)
r7 = Dense(units=3, activation='softmax', name=output_names[5])(avg_pool)
r8 = Dense(units=3, activation='softmax', name=output_names[6])(avg_pool)
r9 = Dense(units=3, activation='softmax', name=output_names[7])(avg_pool)
r10 = Dense(units=3, activation='softmax', name=output_names[8])(avg_pool)
r11 = Dense(units=3, activation='softmax', name=output_names[9])(avg_pool)
r12 = Dense(units=3, activation='softmax', name=output_names[10])(avg_pool)
r13 = Dense(units=3, activation='softmax', name=output_names[11])(avg_pool)
r14 = Dense(units=3, activation='softmax', name=output_names[12])(avg_pool)
r15 = Dense(units=3, activation='softmax', name=output_names[13])(avg_pool)
r16 = Dense(units=3, activation='softmax', name=output_names[14])(avg_pool)
r17 = Dense(units=3, activation='softmax', name=output_names[15])(avg_pool)
r18 = Dense(units=3, activation='softmax', name=output_names[16])(avg_pool)
r19 = Dense(units=3, activation='softmax', name=output_names[17])(avg_pool)
r20 = Dense(units=3, activation='softmax', name=output_names[18])(avg_pool)
r21 = Dense(units=3, activation='softmax', name=output_names[19])(avg_pool)
r22 = Dense(units=3, activation='softmax', name=output_names[20])(avg_pool)
r23 = Dense(units=3, activation='softmax', name=output_names[21])(avg_pool)
r24 = Dense(units=3, activation='softmax', name=output_names[22])(avg_pool)

model = Model(inputs=inputs, 
              outputs=[r2, r3, r4, r5, r6, r7, r8, r9, r10, r11, r12, r13, r14, r15, r16, r17, r18, r19, r20, r21, r22, r23, r24], 
              name='icaonet')

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             (None, 7, 7, 1024)   0                                            
__________________________________________________________________________________________________
global_average_pooling2d_1 (Glo (None, 1024)         0           inputs[0][0]                     
__________________________________________________________________________________________________
blurred (Dense)                 (None, 3)            3075        global_average_pooling2d_1[0][0] 
__________________________________________________________________________________________________
looking_away (Dense)            (None, 3)            3075        global_average_pooling2d_1[0][0] 
__________________________________________________________________________________________________
ink_marked

In [8]:
hist = model.fit(train_features, 
                 np.hsplit(y_train, range(1, y_train.shape[1])), 
                 batch_size=32, 
                 epochs=30, 
                 validation_data=(val_features, np.hsplit(y_val, range(1, y_val.shape[1]))))

save_history(hist.history, 'images/history.pdf')
history_to_dataframe(hist.history)

Train on 5211 samples, validate on 565 samples
Epoch 1/30
5211/5211 [==============================] - 7s 1ms/step - loss: 8.8291 - blurred_loss: 0.4787 - looking_away_loss: 0.7434 - ink_marked_creased_loss: 0.1437 - unnatural_skin_tone_loss: 0.2835 - too_dark_light_loss: 0.2807 - washed_out_loss: 0.0227 - pixelation_loss: 0.3101 - hair_across_eyes_loss: 0.3446 - eyes_closed_loss: 0.5849 - varied_background_loss: 0.3406 - roll_pitch_yaw_loss: 0.6504 - flash_reflection_on_skin_loss: 0.4828 - red_eyes_loss: 0.5079 - shadows_behind_head_loss: 0.6596 - shadows_across_face_loss: 0.4048 - dark_tinted_lenses_loss: 0.2680 - flash_reflection_on_lenses_loss: 0.4214 - frames_too_heavy_loss: 0.2766 - frame_covering_eyes_loss: 0.5426 - hat_cap_loss: 0.3630 - veil_over_face_loss: 0.1190 - mouth_open_loss: 0.5604 - presence_of_other_faces_or_toys_loss: 0.0397 - blurred_acc: 0.8375 - looking_away_acc: 0.7020 - ink_marked_creased_acc: 0.9438 - unnatural_skin_tone_acc: 0.9207 - too_dark_light_acc: 0.932

Epoch 4/30
5211/5211 [==============================] - 4s 712us/step - loss: 5.8735 - blurred_loss: 0.3323 - looking_away_loss: 0.5808 - ink_marked_creased_loss: 0.0941 - unnatural_skin_tone_loss: 0.1721 - too_dark_light_loss: 0.1582 - washed_out_loss: 0.0068 - pixelation_loss: 0.2141 - hair_across_eyes_loss: 0.1921 - eyes_closed_loss: 0.4137 - varied_background_loss: 0.1897 - roll_pitch_yaw_loss: 0.5255 - flash_reflection_on_skin_loss: 0.3053 - red_eyes_loss: 0.3276 - shadows_behind_head_loss: 0.4748 - shadows_across_face_loss: 0.2601 - dark_tinted_lenses_loss: 0.1484 - flash_reflection_on_lenses_loss: 0.2734 - frames_too_heavy_loss: 0.1716 - frame_covering_eyes_loss: 0.3524 - hat_cap_loss: 0.1810 - veil_over_face_loss: 0.0583 - mouth_open_loss: 0.4241 - presence_of_other_faces_or_toys_loss: 0.0171 - blurred_acc: 0.8793 - looking_away_acc: 0.7730 - ink_marked_creased_acc: 0.9578 - unnatural_skin_tone_acc: 0.9422 - too_dark_light_acc: 0.9501 - washed_out_acc: 0.9990 - pixelation_acc: 

5211/5211 [==============================] - 4s 718us/step - loss: 5.4520 - blurred_loss: 0.3022 - looking_away_loss: 0.5628 - ink_marked_creased_loss: 0.0847 - unnatural_skin_tone_loss: 0.1536 - too_dark_light_loss: 0.1384 - washed_out_loss: 0.0048 - pixelation_loss: 0.1995 - hair_across_eyes_loss: 0.1691 - eyes_closed_loss: 0.3871 - varied_background_loss: 0.1699 - roll_pitch_yaw_loss: 0.5017 - flash_reflection_on_skin_loss: 0.2841 - red_eyes_loss: 0.3122 - shadows_behind_head_loss: 0.4321 - shadows_across_face_loss: 0.2430 - dark_tinted_lenses_loss: 0.1392 - flash_reflection_on_lenses_loss: 0.2574 - frames_too_heavy_loss: 0.1586 - frame_covering_eyes_loss: 0.3274 - hat_cap_loss: 0.1603 - veil_over_face_loss: 0.0495 - mouth_open_loss: 0.4005 - presence_of_other_faces_or_toys_loss: 0.0140 - blurred_acc: 0.8914 - looking_away_acc: 0.7762 - ink_marked_creased_acc: 0.9635 - unnatural_skin_tone_acc: 0.9468 - too_dark_light_acc: 0.9536 - washed_out_acc: 0.9990 - pixelation_acc: 0.9340 - ha

Epoch 8/30
5211/5211 [==============================] - 4s 718us/step - loss: 5.0853 - blurred_loss: 0.2847 - looking_away_loss: 0.5342 - ink_marked_creased_loss: 0.0733 - unnatural_skin_tone_loss: 0.1441 - too_dark_light_loss: 0.1238 - washed_out_loss: 0.0033 - pixelation_loss: 0.1926 - hair_across_eyes_loss: 0.1562 - eyes_closed_loss: 0.3599 - varied_background_loss: 0.1582 - roll_pitch_yaw_loss: 0.4864 - flash_reflection_on_skin_loss: 0.2751 - red_eyes_loss: 0.2882 - shadows_behind_head_loss: 0.4053 - shadows_across_face_loss: 0.2219 - dark_tinted_lenses_loss: 0.1218 - flash_reflection_on_lenses_loss: 0.2381 - frames_too_heavy_loss: 0.1413 - frame_covering_eyes_loss: 0.2943 - hat_cap_loss: 0.1448 - veil_over_face_loss: 0.0434 - mouth_open_loss: 0.3832 - presence_of_other_faces_or_toys_loss: 0.0111 - blurred_acc: 0.8985 - looking_away_acc: 0.7927 - ink_marked_creased_acc: 0.9685 - unnatural_skin_tone_acc: 0.9484 - too_dark_light_acc: 0.9585 - washed_out_acc: 0.9990 - pixelation_acc: 

Epoch 11/30
5211/5211 [==============================] - 4s 711us/step - loss: 4.7552 - blurred_loss: 0.2683 - looking_away_loss: 0.5133 - ink_marked_creased_loss: 0.0670 - unnatural_skin_tone_loss: 0.1325 - too_dark_light_loss: 0.1087 - washed_out_loss: 0.0022 - pixelation_loss: 0.1795 - hair_across_eyes_loss: 0.1366 - eyes_closed_loss: 0.3354 - varied_background_loss: 0.1439 - roll_pitch_yaw_loss: 0.4644 - flash_reflection_on_skin_loss: 0.2566 - red_eyes_loss: 0.2709 - shadows_behind_head_loss: 0.3828 - shadows_across_face_loss: 0.2167 - dark_tinted_lenses_loss: 0.1057 - flash_reflection_on_lenses_loss: 0.2218 - frames_too_heavy_loss: 0.1301 - frame_covering_eyes_loss: 0.2790 - hat_cap_loss: 0.1255 - veil_over_face_loss: 0.0391 - mouth_open_loss: 0.3672 - presence_of_other_faces_or_toys_loss: 0.0084 - blurred_acc: 0.9042 - looking_away_acc: 0.8006 - ink_marked_creased_acc: 0.9712 - unnatural_skin_tone_acc: 0.9514 - too_dark_light_acc: 0.9632 - washed_out_acc: 0.9994 - pixelation_acc:

...

5211/5211 [==============================] - 4s 707us/step - loss: 3.9936 - blurred_loss: 0.2230 - looking_away_loss: 0.4595 - ink_marked_creased_loss: 0.0500 - unnatural_skin_tone_loss: 0.1011 - too_dark_light_loss: 0.0844 - washed_out_loss: 4.2934e-04 - pixelation_loss: 0.1549 - hair_across_eyes_loss: 0.1018 - eyes_closed_loss: 0.2828 - varied_background_loss: 0.1177 - roll_pitch_yaw_loss: 0.4202 - flash_reflection_on_skin_loss: 0.2128 - red_eyes_loss: 0.2354 - shadows_behind_head_loss: 0.3322 - shadows_across_face_loss: 0.1705 - dark_tinted_lenses_loss: 0.0818 - flash_reflection_on_lenses_loss: 0.1916 - frames_too_heavy_loss: 0.0977 - frame_covering_eyes_loss: 0.2275 - hat_cap_loss: 0.0970 - veil_over_face_loss: 0.0265 - mouth_open_loss: 0.3230 - presence_of_other_faces_or_toys_loss: 0.0018 - blurred_acc: 0.9173 - looking_away_acc: 0.8231 - ink_marked_creased_acc: 0.9802 - unnatural_skin_tone_acc: 0.9622 - too_dark_light_acc: 0.9699 - washed_out_acc: 0.9998 - pixelation_acc: 0.9434 

Epoch 25/30
5211/5211 [==============================] - 4s 718us/step - loss: 3.8660 - blurred_loss: 0.2134 - looking_away_loss: 0.4508 - ink_marked_creased_loss: 0.0478 - unnatural_skin_tone_loss: 0.0921 - too_dark_light_loss: 0.0759 - washed_out_loss: 4.6908e-04 - pixelation_loss: 0.1500 - hair_across_eyes_loss: 0.0977 - eyes_closed_loss: 0.2737 - varied_background_loss: 0.1190 - roll_pitch_yaw_loss: 0.4153 - flash_reflection_on_skin_loss: 0.2037 - red_eyes_loss: 0.2329 - shadows_behind_head_loss: 0.3148 - shadows_across_face_loss: 0.1638 - dark_tinted_lenses_loss: 0.0789 - flash_reflection_on_lenses_loss: 0.1836 - frames_too_heavy_loss: 0.0932 - frame_covering_eyes_loss: 0.2246 - hat_cap_loss: 0.0915 - veil_over_face_loss: 0.0246 - mouth_open_loss: 0.3171 - presence_of_other_faces_or_toys_loss: 0.0013 - blurred_acc: 0.9207 - looking_away_acc: 0.8254 - ink_marked_creased_acc: 0.9810 - unnatural_skin_tone_acc: 0.9641 - too_dark_light_acc: 0.9714 - washed_out_acc: 1.0000 - pixelation_

5211/5211 [==============================] - 4s 706us/step - loss: 3.7788 - blurred_loss: 0.2101 - looking_away_loss: 0.4365 - ink_marked_creased_loss: 0.0447 - unnatural_skin_tone_loss: 0.0921 - too_dark_light_loss: 0.0721 - washed_out_loss: 2.0941e-04 - pixelation_loss: 0.1537 - hair_across_eyes_loss: 0.0926 - eyes_closed_loss: 0.2660 - varied_background_loss: 0.1135 - roll_pitch_yaw_loss: 0.4061 - flash_reflection_on_skin_loss: 0.2024 - red_eyes_loss: 0.2244 - shadows_behind_head_loss: 0.3061 - shadows_across_face_loss: 0.1613 - dark_tinted_lenses_loss: 0.0759 - flash_reflection_on_lenses_loss: 0.1836 - frames_too_heavy_loss: 0.0955 - frame_covering_eyes_loss: 0.2131 - hat_cap_loss: 0.0911 - veil_over_face_loss: 0.0215 - mouth_open_loss: 0.3152 - presence_of_other_faces_or_toys_loss: 9.8170e-04 - blurred_acc: 0.9250 - looking_away_acc: 0.8309 - ink_marked_creased_acc: 0.9835 - unnatural_skin_tone_acc: 0.9655 - too_dark_light_acc: 0.9745 - washed_out_acc: 1.0000 - pixelation_acc: 0.9

5211/5211 [==============================] - 4s 709us/step - loss: 3.7336 - blurred_loss: 0.2025 - looking_away_loss: 0.4400 - ink_marked_creased_loss: 0.0438 - unnatural_skin_tone_loss: 0.0884 - too_dark_light_loss: 0.0708 - washed_out_loss: 2.1991e-04 - pixelation_loss: 0.1472 - hair_across_eyes_loss: 0.0891 - eyes_closed_loss: 0.2674 - varied_background_loss: 0.1128 - roll_pitch_yaw_loss: 0.4080 - flash_reflection_on_skin_loss: 0.2035 - red_eyes_loss: 0.2256 - shadows_behind_head_loss: 0.3046 - shadows_across_face_loss: 0.1552 - dark_tinted_lenses_loss: 0.0748 - flash_reflection_on_lenses_loss: 0.1768 - frames_too_heavy_loss: 0.0866 - frame_covering_eyes_loss: 0.2124 - hat_cap_loss: 0.0858 - veil_over_face_loss: 0.0235 - mouth_open_loss: 0.3137 - presence_of_other_faces_or_toys_loss: 0.0011 - blurred_acc: 0.9267 - looking_away_acc: 0.8286 - ink_marked_creased_acc: 0.9825 - unnatural_skin_tone_acc: 0.9670 - too_dark_light_acc: 0.9747 - washed_out_acc: 1.0000 - pixelation_acc: 0.9449 

,loss_train,loss_val,acc_train,acc_val
blurred,0.203703,0.332623,92.362311,86.548673
looking_away,0.448661,0.746433,82.671272,78.761062
ink_marked_creased,0.044542,0.152446,98.388025,91.327434
unnatural_skin_tone,0.082845,2.137874,96.967952,73.097345
too_dark_light,0.068234,0.430596,97.562848,88.141593
washed_out,0.000153,0.917637,100.000000,93.982301
pixelation,0.145936,0.216875,94.741892,95.221239
hair_across_eyes,0.089321,0.882789,97.025523,90.265487
eyes_closed,0.261642,0.589228,90.558434,86.725664
varied_background,0.106035,1.581714,96.545769,83.185841


In [9]:
y_trues = np.hsplit(y_val, range(1, y_val.shape[1]))
y_preds = model.predict(val_features)

save_heatmaps(y_trues, y_preds, output_file='images/heatmaps.pdf')